# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('datasets/data.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 колонок и 21525 строк.  
  
В материалах к заданию описаны значения столбцов:
* `children` — количество детей в семье - **int**
* `days_employed` — общий трудовой стаж в днях - **float**
* `dob_years` — возраст клиента в годах - **int**
* `education` — уровень образования клиента - **object**
* `education_id` — идентификатор уровня образования - **int**
* `family_status` — семейное положение - **object**
* `family_status_id` — идентификатор семейного положения - **int**
* `gender` — пол клиента - **object**
* `income_type` — тип занятости - **object**
* `debt` — имел ли задолженность по возврату кредитов - **int**
* `total_income` — ежемесячный доход - **float**
* `purpose` — цель получения кредита - **object**  
  

В названиях колонок нарушений стиля не наблюдается.  
  
В столбцах `days_employed` и `total_income` меньшее кол-во строк по сравнению с остальной таблицей. В них могут присутствовать пропущенные значения.  

In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

В каждой строке таблицы - данные о клиенте банка. В таблице указана личная информация о клиенте:  
* кол-во детей
* уровень образования
* возраст
* семейное положение
* пол  
  
Так же указана финансовая активность клиента:  
* общий трудовой стаж
* тип занятости
* была ли задолжность по возрату кредита
* ежемесячный доход
* цель получения кредита  
  
Данных достаточно для проверки гипотез о влиянии семейного положения и кол-ва детей.  
На первый взгляд потребуется обработка данных:  
1. решить вопросы с пустыми значениями
2. изменить тип данных в столбцах `days_employed` и `total_income` на **int**
3. убрать дубликаты в `education`
4. просмотреть список значений в столбце `purpose`, отвечающем за цель кридита и применить к ним лемматизацию
5. применить категоризацию к таблице, выделив из нее данные для создания словарей на основе столбцов `family_status` и `education`

## Шаг 2. Предобработка данных

### Обработка пропусков

Для начала узнаем в каких столбцах присутствуют пропущенные значения.

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах `days_employed` и `total_income` есть пропущенные значение и их равное кол-во. Учитывая, что эти столбцы отвечают за **трудовой стаж** и **ежемесячный доход**, возможно в этих строках указаны безработные, индивидуальные предприниматели или студенты.    
Проверим 2 гипотезы:  
* клиенты без стажа и клиенты без ежемесячного дохода - это одни и те же клиенты
* клиенты без стажа и ежемесячного дохода - это студенты, индивидуальные предприниматели или безработные

In [5]:
df_dropna = df.dropna(subset=['days_employed'], inplace=False)

In [6]:
df_dropna.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Гипотеза подтвердилась**, пропущенные значения в столбцах `days_employed` и `total_income` относятся к одним и тем же строкам, но удалять эти строки не имеет смысла, т.к. они **не критичны для проверки 2х основных гипотез этого проекта**. По этой причине заполним их значением `0`, которое вполне соответствует типу данных в столбцах.  
Но нам придется учитывать этот `0` при проверке третьей гипотезы, касающейся уровня дохода.

In [7]:
df = df.fillna(0)
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [8]:
df_no_income = df[df['days_employed'] == 0]
df_no_income.groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,147,147,147,147,147,147,147,147,147,147,147
компаньон,508,508,508,508,508,508,508,508,508,508,508
пенсионер,413,413,413,413,413,413,413,413,413,413,413
предприниматель,1,1,1,1,1,1,1,1,1,1,1
сотрудник,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105,1105


**Гипотеза не подтвердилась**, клиенты без стажа и ежемесячного дохода имеют разный тип занятости и данные о них могли быть утеряны, либо не подтверждены нужными документами.

**Вывод**

Пропущенные значения были обнаружены в столбцах `days_employed` и `total_income`. Мы выдвинули две гипотезы:  
* клиенты без стажа и клиенты без ежемесячного дохода - это одни и те же клиенты
* клиенты без стажа и ежемесячного дохода - это студенты, индивидуальные предприниматели или безработные  
  
Первая гипотеза подтвердилась - это действительно одни и те же строки, вторая была опровергнута - тип занятости у клиентов с незаполненными данными не позволяет сделать конкретных выводов и скорее всего является случайной ошибкой.  
  
Пустые значения в столбцах были заменены на `0`, т.к. их содержание не критично для проверки основной гипотезы, при этом полностью соответствует типу данных в столбцах.

### Замена типа данных

Столбцы, в которых мы искали пропущенные значения `days_employed` и `total_income`, имеют тип данных **float**, который противоречит хранящейся в них информации.   
* Количество рабочих дней не может быть дробным или отрицательным.
* Ежемесячный доход так же можно округлить до целых чисел.

Для смены типа данных на **int** лучше всего подойдет метод astype().

In [9]:
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

In [10]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Тип данных в столбцах был изменен, но на этом проблемы со столбцом `days_employed` не кончились.  
В глаза брасаются еще 2 момента:  
* отрицательные значения
* слишком большие значения

Начнем с того, что уберем отрицательные значения, скорее всего это просто обычные значения, записанные по каким то причинам со знаком минус. Об этом можно судить по верхней части таблицы, где мы видим, что чем старше клиент, тем больше значение в трудовом стаже, не смотря на знак минус.

In [11]:
df['days_employed'] = df['days_employed'].abs()

In [12]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Теперь попробуем разобраться со слишком большими значениями в столбце `days_employed`.  
Учитывая, что расчет идет в днях, попробуем взять за основу, что человек может иметь рабочий стаж свыше 60 лет в крайне редких случаях.  
`60 \* 365 = 21 900`  
Округлим это значение до `22 000` и будем считать его максимальным допустимым стажем.  
Посмотрим сколько клиентов имеют стаж выже допустимого.

In [13]:
df_long_experience = df[df['days_employed']>22000]
print(df_long_experience['days_employed'].count())

3445


Группируем этих клиентов по типу занятости.

In [14]:
df_long_experience.groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
безработный,2,2,2,2,2,2,2,2,2,2,2
пенсионер,3443,3443,3443,3443,3443,3443,3443,3443,3443,3443,3443


Большая часть клиентов относится к пенсионерам, мы можем смело поставить им максимальный трудовой стаж. Безработным поставим 0, их всего 2 и это не повлияет на общую статистику.

In [15]:
df.loc[(df['days_employed']>22000) & (df['income_type']=='пенсионер'), 'days_employed'] = 22000
df.loc[(df['days_employed']>22000) & (df['income_type']=='безработный'), 'days_employed'] = 0
df['days_employed'].max()

22000

In [16]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,22000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Тип данных в столбцах `days_employed` и `total_income` был заменен с **float** на **int**. Заодно мы немного подправили столбец `days_employed`, поменяв отрицательные значения на положительные и убрав слишком большой трудовой стаж.  

### Обработка дубликатов

Сперва справим все неявные дубликаты, т.к. значения в столбце `education` могут быть записаны по разному, они могут не проявиться при поиске явных дубликатов. После чего займемся явными.  
Проверим какой список уникальных значений есть в столбцах.

In [17]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

`education` - потребуется привести все значения к нижнему регистру. Причина появления дубликатов может крыться в человеческом факторе и отсутствии стандартов заполнения форм. В дальнейшем это можно исключить приводя данные к нижнему регистру при записи.

In [18]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

`family_status` - так же все буквы сделаем строчными.

In [19]:
df['education_id'].unique()

array([0, 1, 2, 3, 4])

`education_id` - преобразований не требуется.

In [20]:
df['family_status_id'].unique()

array([0, 1, 2, 3, 4])

`family_status_id` - преобразований не требуется.

In [21]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

`gender` - преобразований не треубется, дубликатов нет.

In [22]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

`income_type` - преобразований не треубется, дубликатов нет.

In [23]:
df['debt'].unique()

array([0, 1])

`debt` - преобразований не треубется, дубликатов нет.

In [24]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

`purpose` - здесь очень много разных значений, столбец требует лемматизации.

In [25]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

`children` - отрицательные значения заменим на 0, а значение в 20 скорре всего является опечаткой, его мы заменим на 2.

По итогу проверки удаление дубликатов требует столбец `education`.   <br>
Столбец `family_status` нужно привести буквы к строчной форме. <br>
Столбец `children` убрать отрицательные и слишком большие значения.

In [26]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [27]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Вроде все получилось. <br>  Для проверки сравним кол-во уникальных значений `education` и `family_status` с колчиством уникальных значений соответствующих `ID`.

In [28]:
len(df['education'].unique()) == len(df['education_id'].unique())

True

In [29]:
len(df['family_status'].unique()) == len(df['family_status_id'].unique())

True

Остался столбец `children`. Удалим отрицательные значения и слишком большое значение 20 заменим на 2.

In [30]:
df.loc[df['children'] < 0, 'children'] = 0
df.loc[df['children'] == 20, 'children'] = 2
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

После того, как мы удалили неявные дубликаты, удалим явные.

In [31]:
df.duplicated().sum()

71

In [32]:
df = df.drop_duplicates().reset_index(drop=True)

In [33]:
df.duplicated().sum()

0

Явных дубликатов в таблице не осталось.

**Вывод**

Дубликаты в `education` были удалены приведением значений столбца к нижнему регистру. <br>  Требуется сообщить команде разработки о необходимости приведения данных, попадающих в этот столбец к нижнему регистру при сохранении.
  
`family_status` так же был приведен к нижнему регистру. <br> Требуется сообщить команде разработки о необходимости приведения данных, попадающих в этот столбец к нижнему регистру при сохранении.  

В столбце `children` были изменены ошибочные значения -1 и 20 на 0 и 2.  
  
После чего мы удалили явные дубликаты.

### Лемматизация

Последний столбец `purpose`, содержащий в себе цели взятия кредита содержит большое кол-во уникальных записей. Попробуем лемматизировать его и понять сколько же реальных целей существует при взятии кредита.

Для начала подключим библиотеку.

In [34]:
from pymystem3 import Mystem
m = Mystem()

Далее передадим ей список уникальных значений из столбца `purpose`. Учитывая, что для лемматизации требуется текст, преобразуем список в текст.

In [35]:
purpose_list = df['purpose'].unique()
purpose_list = " ".join(purpose_list)
lemmas = m.lemmatize(purpose_list)
lemmas

['покупка',
 ' ',
 'жилье',
 ' ',
 'приобретение',
 ' ',
 'автомобиль',
 ' ',
 'дополнительный',
 ' ',
 'образование',
 ' ',
 'сыграть',
 ' ',
 'свадьба',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'жилье',
 ' ',
 'образование',
 ' ',
 'на',
 ' ',
 'проведение',
 ' ',
 'свадьба',
 ' ',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'для',
 ' ',
 'семья',
 ' ',
 'покупка',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'собственный',
 ' ',
 'недвижимость',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'недвижимость',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'подержать',
 ' ',
 'автомобиль',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'свой',
 ' ',
 'автомобиль',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'жилье',
 ' ',
 'операция',
 ' ',
 'со',
 ' ',
 'свой',
 ' ',
 'недвижимо

Посчитаем сколько раз каждое слово повторяется в списке при помощи библиотеки `collections` и ее  класса `Counter`.

In [36]:
from collections import Counter

In [37]:
lemmas_counter_list = Counter(lemmas)

Сразу сортируем список для наглядности.

In [38]:
lemmas_counter_list.most_common()

[(' ', 96),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('подержать', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('со', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('\n', 1)]

**Вывод**

Лемматизация столбца, содержащего цели кредита выявила, что основными целями являются дорогостоящие покупки и траты.  
Самые распространненые из них:  
* покупка недвижимости
* покупка автомобиля
* оплата обучения
* операция
* свадьба

### Категоризация данных

Посмотрим еще раз на нашу таблицу.

In [39]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,22000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


В таблице присутствуют столбцы `education` и `family_status`, а так же их ID `education_id` и `family_status_id`.  
Перенесем столбцы `education` и `family_status` в словари для увеличения скорости работы с таблицей.

In [40]:
cleaned_df = df[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id',
                 'gender', 'income_type', 'debt', 'total_income', 'purpose']]
education_dict = df[['education_id', 'education']]
family_status_dict = df[['family_status_id', 'family_status']]

Проверим новые таблицы и удалим из словарей явные дубликаты.

In [41]:
cleaned_df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,22000,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


In [42]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [43]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


**Вывод**

На основе базовой таблицы `df` была создана новая `cleaned_df` и два словаря к ней `education_dict` и `family_status_dict`.  
В новом очищенном виде таблица готова к проверке основных гипотез.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для проверки данной гипотезы нам потребуется понять как наличие детей в столбце `children` влияет на значение в столбце `debt`, отвечающем за долг.  
Для этого сгруппируем методом `grouopby` таблицу по кол-ву детей и посчитаем отношение общего кол-ва клиентов к количеству должников по каждой категории.

Найдем общее кол-во клиентов по каждой категории.

**Альтернативный вариант решения, общий для всех вопросов.**

In [44]:
def debt_conversion(hypothesis):
    testing_data = df[[hypothesis, 'debt']]
    
    testing_pivot = testing_data.pivot_table(index=hypothesis, values='debt', aggfunc=['sum', 'count'])
    testing_pivot['conversion'] = (testing_pivot['sum'] / testing_pivot['count']).round(decimals=2)
    
    return testing_pivot


children_test = debt_conversion('children')
children_test

,sum,count,conversion
,debt,debt,
children,,,
0,1064,14138,0.08
1,444,4808,0.09
2,202,2128,0.09
3,27,330,0.08
4,4,41,0.10
5,0,9,0.00


In [45]:
children_debt_count = cleaned_df.groupby('children')['debt'].count()
children_debt_count

children
0    14138
1     4808
2     2128
3      330
4       41
5        9
Name: debt, dtype: int64

Теперь общее кол-во должников по каждой категории.

In [46]:
children_debt_sum = cleaned_df.groupby('children')['debt'].sum()
children_debt_sum

children
0    1064
1     444
2     202
3      27
4       4
5       0
Name: debt, dtype: int64

Теперь объединим обе серии методом concat().

In [47]:
children_debt = pd.concat([children_debt_count, children_debt_sum], axis=1)
children_debt.columns = ['count', 'sum']
children_debt

,count,sum
children,,
0,14138,1064
1,4808,444
2,2128,202
3,330,27
4,41,4
5,9,0


Теперь осталось рассчитать отношение клиентов с долгами к общему кол-ву клиентов и записать это в новый столбец.  
Для начала напишем функцию, которую мы будем использовать для создания нового столбца.

In [48]:
def part_of_debt(row):
    c_sum = row['sum']
    c_count = row['count']
    
    if c_sum == 0:
        return 0
    else:
        return c_sum / c_count

Осталось создать новый столбец.

In [49]:
children_debt['part'] = children_debt.apply(part_of_debt, axis=1)
children_debt

,count,sum,part
children,,,
0,14138,1064,0.075258
1,4808,444,0.092346
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


Сократим количество точек после запятой в столбце `part` для наглядности.

In [50]:
children_debt['part'] = children_debt['part'].round(decimals=2)
children_debt

,count,sum,part
children,,,
0,14138,1064,0.08
1,4808,444,0.09
2,2128,202,0.09
3,330,27,0.08
4,41,4,0.10
5,9,0,0.00


**Вывод**

Судя по всему наличие детей не влияет на своевременный возврат кредита заемщиком.  
Гипотеза оказалась неверной.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для изучения данной гипотезы используем ту же модель, что и в прошлый раз.  
Сперва найдем кол-во клиентов в каждой категории семейного положения.

In [51]:
family_debt_count = cleaned_df.groupby('family_status_id')['debt'].count()

Потом найдем сумму просроченных кредитов по каждой категории.

In [52]:
family_debt_sum = cleaned_df.groupby('family_status_id')['debt'].sum()

Далее объединим все в одну таблицу и переименем названия столбцов.

In [53]:
family_debt = pd.concat([family_debt_count, family_debt_sum], axis=1)

In [54]:
family_debt.columns = ['count', 'sum']

Объединим таблицу со словарем по ID.

In [55]:
family_debt = family_debt.merge(family_status_dict, on ='family_status_id', how='left')

In [56]:
family_debt

,family_status_id,count,sum,family_status
0,0,12339,931,женат / замужем
1,1,4151,388,гражданский брак
2,2,959,63,вдовец / вдова
3,3,1195,85,в разводе
4,4,2810,274,не женат / не замужем


Добавим новый столбец, применив формулу, созданную для прошлой гипотезы.

In [57]:
family_debt['part'] = family_debt.apply(part_of_debt, axis=1)
family_debt

,family_status_id,count,sum,family_status,part
0,0,12339,931,женат / замужем,0.075452
1,1,4151,388,гражданский брак,0.093471
2,2,959,63,вдовец / вдова,0.065693
3,3,1195,85,в разводе,0.071130
4,4,2810,274,не женат / не замужем,0.097509


In [58]:
family_debt['part'] = family_debt['part'].round(decimals=2)
family_debt

,family_status_id,count,sum,family_status,part
0,0,12339,931,женат / замужем,0.08
1,1,4151,388,гражданский брак,0.09
2,2,959,63,вдовец / вдова,0.07
3,3,1195,85,в разводе,0.07
4,4,2810,274,не женат / не замужем,0.10


**Вывод**

Семейное положение не оказывает значительное влияние на возврат кредитов. Разбег между категориями составляет несколько процентов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Посмотрим еще раз как выглядит таблица.

In [59]:
cleaned_df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,22000,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


Проверим минимальные и максимальные значения столбца `total_income`.

In [60]:
cleaned_df['total_income'].min()

0

In [61]:
cleaned_df['total_income'].max()

2265604

Очевидно, что при 0 значении нам не известен доход клиента, поэтому надо будет выделить этих клиентов в отдельную группу.  
Учитывая, что клиенты очень сильно разделяются по доходу, наилучшим решением будет группировать их по уровню дохода:
* доход равный 0 - неизвестен
* от 0 до 50 000 - низкий доход
* от 50 000 до 150 000 - средний доход
* от 150 000 до 500 000 - высокий доход
* от 500 и выше - список Forbs  
  
Для этого сперва напишем функцию.

In [62]:
def income_split(income):
    
    if income == 0:
        return 'доход неизвестен'
    if income < 50000:
        return 'низкий доход'
    if income < 150000:
        return 'средний доход'
    if income < 500000:
        return 'высокий доход'
    else:
        return 'список Forbs'

Используя функцию создадим новый столбец методом `apply()`.

In [63]:
cleaned_df['income_category'] = cleaned_df['total_income'].apply(income_split)

<ipython-input-63-3ed8f9889262>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['income_category'] = cleaned_df['total_income'].apply(income_split)


Далее группируем значения по новому столбцу и подсчитываем их кол-во и сумму.   
После чего объединяем вместе как при прошлых гипотезах.

In [64]:
income_count = cleaned_df.groupby('income_category')['debt'].count()

In [65]:
income_sum = cleaned_df.groupby('income_category')['debt'].sum()

In [66]:
income = pd.concat([income_count, income_sum], axis=1)

In [67]:
income.columns = ['count', 'sum']
income

,count,sum
income_category,,
высокий доход,8962,712
доход неизвестен,2103,170
низкий доход,372,23
список Forbs,222,14
средний доход,9795,822


Применяем формулу из первой гипотезы для расчета отношения и сокращаем кол-во знаков после запятой в получившихся столбцах.

In [68]:
income['part'] = income.apply(part_of_debt, axis=1)
income['part'] = income['part'].round(decimals=2)
income

,count,sum,part
income_category,,,
высокий доход,8962,712,0.08
доход неизвестен,2103,170,0.08
низкий доход,372,23,0.06
список Forbs,222,14,0.06
средний доход,9795,822,0.08


**Вывод**

Большая часть клиентов имеет доход от 50 000 до 500 000 и кол-во должников из них составляет около 8%.  
Чуть лучше выплачивают кредиты клиенты с доходом ниже 50 000 или выше 500 000, среди них должники составляют около 6%.
Судя по всему своевременная оплата кредитов не сильно зависит от уровня дохода клиента, а скорее является чертой характера.

- Как разные цели кредита влияют на его возврат в срок?

Для ответа на этот вопрос сперва вспомним список целей кредита после лематизации.

In [69]:
lemmas_counter_list

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

На основании этого словаря составим свой собственный словарь, куда включим основные причины взятия кредита.

In [70]:
purpose_dict = {
    'недвижимость': ['жилье', 'недвижимость', 'строительство', 'жилой', 'коммерческий', 'строительство'],
    'автомобиль': ['автомобиль', 'подержать'],
    'образование': ['образование'],
    'свадьба': ['свадьба'],
}

Теперь напишем функцию, которая будет сравнивать лемматизированные значения в столбце `pupose` и наш новый словарь и добавлять в новый столбец `purpose_lem` подходящее значение.

In [71]:
def copmpare_purpose_and_dict(purpose):
    lemm_purpose = m.lemmatize(purpose)
    for word in purpose_dict:
        for value in purpose_dict[word]:
            if value in lemm_purpose:
                return word
    
    return 'нет значения'

Теперь применим функцию к столбцу `purpose` и создадим новый столбец `purpose_lem`.

In [72]:
cleaned_df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,высокий доход
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,средний доход
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,средний доход
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,высокий доход
4,0,22000,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,высокий доход


In [73]:
cleaned_df['purpose_lem'] = cleaned_df['purpose'].apply(copmpare_purpose_and_dict)

Далее действуем по старой схеме: группируем, считаем кол-во и сумму и объединяем в одну таблицу.  
В новой таблице переименовываем столбцы.

In [74]:
purpose_count = cleaned_df.groupby('purpose_lem')['debt'].count()
purpose_sum = cleaned_df.groupby('purpose_lem')['debt'].sum()
purpose = pd.concat([purpose_count, purpose_sum], axis=1)
purpose.columns = ['count', 'sum']
purpose

,count,sum
purpose_lem,,
автомобиль,4306,403
недвижимость,10811,782
образование,4013,370
свадьба,2324,186


К новой таблице применяем старую функцию `part_of_debt` и записываем результат в новый столбец `part`.  
После чего уменьшаем количество знаков после запятой в столбце до 2х.

In [75]:
purpose['part'] = purpose.apply(part_of_debt, axis=1)
purpose['part'] = purpose['part'].round(decimals=2)
purpose

,count,sum,part
purpose_lem,,,
автомобиль,4306,403,0.09
недвижимость,10811,782,0.07
образование,4013,370,0.09
свадьба,2324,186,0.08


**Вывод**

Проанализировав причины по которым люди берут кредит, мы можем сделать вывод, что больше всего кредитов берется на покупку недвижимости и меньше всего на свадьбу. Где между ними находятся покупка автомобиля и оплата образования. Скорее всего это связано с величиной затрат, так покупка жилья для большинства клиентов является самой большой тратой в жизни, тогда как траты на свадьбу - самой малой.  
  
Лучше всего отдают кредиты связанные с недвижимостью, зодлжность только 7% и хуже всего кредиты на автомобиль и образование - задолжность 9%.

## Шаг 4. Общий вывод

В этом исследовании мы ответили на 4 вопроса:  

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?  
  
По большей части нет, люди с детьми возвращают кредит примерно так же, как и люди без детей.
  
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?  
  
Клиенты из категории не женат / не замужем и находящиеся в гражданском браке отдают кредиты хуже всего.  
Лучше всего отдают кредиты вдовцы и разведенные.  
  
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
  
Клиенты с доходом ниже 50 000 или выше 500 000 возвращают кредит лучше, чем все остальные.  
  
4. Как разные цели кредита влияют на его возврат в срок?

Лучше всего отдают кредиты связанные с недвижимостью, зодлжность только 7% и хуже всего кредиты на автомобиль и образование - задолжность 9%.

Из исследования видно, что все перечисленные факторы не сильно влияют на своевременную оплату кредита клиентом. 